# Modelo de validación
Mida el rendimiento de su modelo para poder probar y comparar alternativas.

Has construido un modelo. ¿Pero qué tan bueno es?

En esta lección, aprenderá a utilizar la validación de modelos para medir la calidad de su modelo. Medir la calidad del modelo es la clave para mejorar iterativamente sus modelos.

### ¿Qué es la validación de modelos?

Querrá evaluar casi todos los modelos que haya construido. En la mayoría de las aplicaciones (aunque no en todas), la medida relevante de la calidad del modelo es la precisión predictiva. En otras palabras, ¿las predicciones del modelo se acercarán a lo que realmente sucede?

Mucha gente comete un gran error al medir la precisión predictiva. Hacen predicciones con sus datos de entrenamiento y las comparan con los valores objetivo en los datos de entrenamiento. Verá el problema con este enfoque y cómo resolverlo en un momento, pero pensemos en cómo haríamos esto primero.

Primero deberá resumir la calidad del modelo de una manera comprensible. Si compara los valores reales y previstos de 10.000 casas, probablemente encontrará una combinación de predicciones buenas y malas. Sería inútil revisar una lista de 10.000 valores previstos y reales. Necesitamos resumir esto en una sola métrica.

Hay muchas métricas para resumir la calidad del modelo, pero comenzaremos con una llamada Error Absoluto Medio (también llamado MAE). Analicemos esta métrica comenzando con la última palabra, error.

El error de predicción para cada casa es:

>> error=actual−predicted

Entonces, si una casa cuesta $150 000 y usted predijo que costaría $100 000, el error es $50 000.

Con la métrica MAE tomamos el valor absoluto de cada error. Esto convierte cada error en un número positivo. Luego tomamos el promedio de esos errores absolutos. Esta es nuestra medida de la calidad del modelo. En lenguaje sencillo, se puede decir como

>> En promedio, nuestras predicciones están equivocadas en aproximadamente X.

In [1]:
# Data Loading Code Hidden Here
import pandas as pd

# Load data
melbourne_file_path = 'melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 
# Filter rows with missing price values
filtered_melbourne_data = melbourne_data.dropna(axis=0)
# Choose target and features
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.tree import DecisionTreeRegressor
# Define model
melbourne_model = DecisionTreeRegressor()
# Fit model
melbourne_model.fit(X, y)

DecisionTreeRegressor()

Una vez que tenemos un modelo, así es como calculamos el error absoluto medio:

In [2]:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

434.71594577146544

### El problema de las puntuaciones "dentro de la muestra"

La medida que acabamos de calcular se puede denominar puntuación "dentro de la muestra". Utilizamos una única "muestra" de casas tanto para construir el modelo como para evaluarlo. He aquí por qué esto es malo.

Imaginemos que, en el gran mercado inmobiliario, el color de las puertas no guarda relación con el precio de la vivienda.

Sin embargo, en la muestra de datos que utilizó para construir el modelo, todas las casas con puertas verdes eran muy caras. El trabajo del modelo es encontrar patrones que predigan los precios de las viviendas, por lo que verá este patrón y siempre predecirá precios altos para las viviendas con puertas verdes.

Dado que este patrón se derivó de los datos de entrenamiento, el modelo aparecerá preciso en los datos de entrenamiento.

Pero si este patrón no se mantiene cuando el modelo ve nuevos datos, el modelo sería muy inexacto cuando se use en la práctica.

Dado que el valor práctico de los modelos proviene de hacer predicciones sobre datos nuevos, medimos el rendimiento de datos que no se utilizaron para construir el modelo. La forma más sencilla de hacer esto es excluir algunos datos del proceso de creación del modelo y luego usarlos para probar la precisión del modelo en datos que no se han visto antes. Estos datos se denominan datos de validación.

### Codificándolo

La biblioteca scikit-learn tiene una función train_test_split para dividir los datos en dos partes. Usaremos algunos de esos datos como datos de entrenamiento para ajustarnos al modelo y usaremos los demás datos como datos de validación para calcular el error_absoluto medio.

Aquí está el código:

In [3]:
from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# Define model
melbourne_model = DecisionTreeRegressor()
# Fit model
melbourne_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

262254.7843770174


### ¡Guau!

Su error absoluto medio para los datos de la muestra fue de unos 500 dólares. Fuera de muestra asciende a más de 250.000 dólares.

Ésta es la diferencia entre un modelo que es casi exactamente correcto y uno que no se puede utilizar para la mayoría de los propósitos prácticos. Como punto de referencia, el valor medio de la vivienda en los datos de validación es de 1,1 millones de dólares. Entonces, el error en los nuevos datos es aproximadamente una cuarta parte del valor promedio de una vivienda.

Hay muchas formas de mejorar este modelo, como experimentar para encontrar mejores características o diferentes tipos de modelos.

# EJERCICIOS

### Resumen
Has construido un modelo. En este ejercicio probarás qué tan bueno es tu modelo.

Ejecute la siguiente celda para configurar su entorno de codificación donde terminó el ejercicio anterior.

In [ ]:
# Code you have previously used to load data
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

# Path of the file to read
iowa_file_path = '../input/home-data-for-ml-course/train.csv'

home_data = pd.read_csv(iowa_file_path)
y = home_data.SalePrice
feature_columns = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = home_data[feature_columns]

# Specify Model
iowa_model = DecisionTreeRegressor()
# Fit Model
iowa_model.fit(X, y)

print("First in-sample predictions:", iowa_model.predict(X.head()))
print("Actual target values for those homes:", y.head().tolist())

### Paso 1: divide tus datos

Utilice la función train_test_split para dividir sus datos.

Déle el argumento random_state=1 para que las funciones de verificación sepan qué esperar al verificar su código.

Recuerde, sus funciones se cargan en DataFrame X y su objetivo se carga en y.

In [ ]:
# Import the train_test_split function and uncomment
# from _ import _
from sklearn.model_selection import train_test_split
# fill in and uncomment
# train_X, val_X, train_y, val_y = ____
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)

### Paso 2: especificar y ajustar el modelo
Cree un modelo DecisionTreeRegressor y ajústelo a los datos relevantes. Establezca random_state en 1 nuevamente al crear el modelo.

In [ ]:
# You imported DecisionTreeRegressor in your last exercise
# and that code has been copied to the setup code above. So, no need to
# import it again

# Specify the model
iowa_model = DecisionTreeRegressor(random_state=1)

# Fit iowa_model with the training data.
iowa_model.fit(train_X, train_y)

### Paso 3: hacer predicciones con datos de validación

In [ ]:
# Predict with all validation observations
val_predictions = iowa_model.predict(val_X)

### Paso 4: Calcule el error absoluto medio en los datos de validación

In [ ]:
from sklearn.metrics import mean_absolute_error
val_mae = mean_absolute_error(val_y, val_predictions)

# uncomment following line to see the validation_mae
print(val_mae)

¿Es bueno ese MAE? No existe una regla general sobre qué valores son buenos que se aplique en todas las aplicaciones. Pero verás cómo usar (y mejorar) este número en el siguiente paso.